## EDP Parabólica - Equação da Condução de Calor Unidimensional
---
### Equação Diferencial
$$\dfrac{\partial T}{\partial t} = \alpha \dfrac{\partial^2 T}{\partial x^2} $$

### MDF - Método Explícito
$$ T^{l+1}_{i} = T^{l}_{i} + \lambda(T^{l}_{i+1} - 2T^{l}_{i} + T^{l}_{i-1}) $$
$$ \lambda = k \cfrac{\Delta t}{(\Delta x)^2} $$
onde $\alpha$ é a difusibilidade térmica do material em m^2/s, $T$ a temperatura em °C, $\Delta t$ é o passo de tempo em s e $\Delta x$ o passo de comprimento em m. $l$ representa o instante e $i$ a posição.

#### MDF - Condição na Derivada (Neumann)
Onde houver uma condição na deririvada ($\frac{\partial T}{\partial x}$), deverá ser criado um ponto imaginário no contorno, que manterá a derivada constante nessa estremidade. Por exemplo, a temperatura no ponto à esquerda da barra será obtida a partir da aproximação da derivada primeira em relação a $x$:
$$ \dfrac{\partial T}{\partial x} \approx \dfrac{\Delta T}{2\Delta x} = \dfrac{T^l_1 - T^l_{-1}}{2\Delta x} $$

Sendo o fluxo dado, a temperatura $T^l_{-1}$ no ponto auxiliar esquerdo será dado por
$$ T^l_{-1} = T^l_1 - 2\Delta x\dfrac{\partial T}{\partial x}$$

Analogamente, no lado direito, tem-se
$$ T^l_{n+1} = T^l_{n-1} + 2\Delta x\dfrac{\partial T}{\partial x}$$

---

In [ ]:
# ----------------------------------------------------------------- #
#                    Condução de Calor Unidimensional               #
#                pelo Método das Diferenças Finitas para            #
#             Condições de Contorno de Dirichlet e Neumann          #
# ----------------------------------------------------------------- #
#                           Pedro F M Pires                         #
#                          Filipe A M Eismann                       #
#                             (out/2020)                            #
# ----------------------------------------------------------------- #

# ================================================================= #
# NOTA:
# ----------------------------------------------------------------- #
# A implemantação do fluxo está um pouco enganosa, pois não obedece 
# a definição de fluxo de calor, mas ainda é aceitavel dar o valor 
# da derivada dT/dx em si, que é mais fácil de verificar graficamente.
# ================================================================= #

# Somente para o jupyter notebook, retirar para rodar o arquivo .py
#%matplotlib notebook 
import numpy as np
import matplotlib.pyplot as plt

# ----------------------------------------------------------------- #
#                            PARÂMETROS                             #
# ----------------------------------------------------------------- #
L = 10                  # [cm] Comprimento da barra
tf = 15                 # [s] Instante final (aproximado)
dx = 2                  # [cm] Passo de distância

# --- Propriedades do Material da Barra
# Aço: rho = 7800 kg/m3, k =  52 W/(m·K), c = 486 J/(kg·K)
#  Al: rho = 2697 kg/m3, k = 237 W/(m·K), c = 900 J/(kg·K)

# rho = 2697;             #    [kg/m3] Massa específica
# k = 237;                #  [W/(m·K)] Condutividade térmica
# c = 900;                # [J/(kg·K)] Capacidade térmica específica
# alpha = k/(rho*c)       #   [m²/(s)] Difusibilidade térmica do material
alpha = 0.49/(2.7*0.2174)#  [cm²/(s)] Difusibilidade térmica do material (dado Chapra, 2016)

# --- Passo de Tempo
#lamb = 1/10;
lamb = 1/6              # Valor mínimo que garante a convergência e estabilidade da solução (Chapra, 2016)
#lamb = 1/2             # Valor mínimo que garante a convergência da solução
dt = lamb*dx**2/alpha   # Passo de em função de lambda

# ----------------------------------------------------------------- #
#                     INICIALIZAÇÃO DOS VETORES                     #
# ----------------------------------------------------------------- #
X = np.arange(0, L+dx, dx)       # Vetor da coordenada X
Inst = np.arange(0, tf+dt, dt)   # Vetor dos instantes t
fluxo = np.zeros(2)              # Vetor das flags de cond. de fluxo

# --- Inicialização da Matriz de Temperaturas
# primeira posição: instante, segunda: coordenada 'x'
T = np.zeros([len(Inst), len(X)]) # T[instante, posição]

# ----------------------------------------------------------------- #
#                          FLAGS DE FLUXO                           #
# ----------------------------------------------------------------- #
Letras = 'd D f F'      # String das letras que indicam cond. de fluxo
letras = Letras.split() # Criando um vetor das letras separadas

# ----------------------------------------------------------------- #
#                         CONDIÇÃO INICIAL                          #
# ----------------------------------------------------------------- #
# Caso a barra estaja pré-aquecida inicializar o vetor com a(s) temperatura(s) inicial(ais)
#for i in range(1,len(X) -1):
#    T[0,i] = 2*X[i]
    
# ----------------------------------------------------------------- #
#                       CONDIÇÕES DE CONTORNO                       #
# ----------------------------------------------------------------- #
# Se for digitada uma das letras 'd' ou 'f' é interpretada uma cond. 
# de fluxo/derivada. Nesses casos insere-se uma nova coluna na primeira 
# ou ultima posição da matriz e a preenche com o valor auxiliar .
# ----------------------------------------------------------------- #

# --- Lado Esquerdo
T_esq = input("Temperatura no lado esquerdo (digite d para cond. na derivada): ")
if T_esq in letras:
    T_esq = float(input("dT/dx no lado esquerdo: "))
    dT = T[0,1]-2*dx*T_esq                # Valor inicial da coluna auxiliar para fixar a derivada
    T = np.insert(T, 0, dT, axis=1) # Insere o valor 'dT' na coluna (axis=1) 0 do vetor 'T'
    fluxo[0] = 1                    # Flag de condição no fluxo
else: 
    # Condição de Temperatura no Lado Esquerdo
    T[:, 0] = float(T_esq)


# --- Lado Direito
T_dir = input(" Temperatura no lado direito (digite d para cond. na derivada): ") 
if T_dir in letras:
    T_dir = float(input("dT/dx no lado direito: "))
    # Valor da coluna auxiliar para fixar a derivada
    dT = T[0,-1]+2*dx*T_dir
    # Insere uma nova coluna na ultima posição da matriz e a preenche com dT
    T = np.insert(T, len(T[0]), dT, axis=1)
    # Flag de condição no fluxo
    fluxo[1] = 1
else: 
    # Condição de Temperatura no Lado Direito
    T[:, -1] = float(T_dir)

# Impressão do vetor da condição inicial
#print("T inicial = ",T[0,:])

# ----------------------------------------------------------------- #
#                              SOLUÇÃO                              #
# ----------------------------------------------------------------- #
nt = len(T[:,0])  # Número de instantes, incluindo as posições de fluxo
nx = len(T[0, :]) # Número de coordenadas, incluindo as posições de fluxo

for t in range(1, nt):       # Percorre cada instante 't' ignorando a posição [0], o tempo não possui condição limitante (aberto no infinito)
    for x in range(1, nx-1): # Percorre cada coordenada 'x' ignorando a posição [0], a coordenada 'x' é limitada pela CC no lado direito, por isso 'nx-1'
        # --- Caso cond. de fluxo no lado esquerdo 
        if fluxo[0] == 1:
            T[t,0] = T[t,2] -2*dx*T_esq
                    
            
        # --- Cálculo das temperaturas no pontos internos
        T[t,x] = T[t-1,x] + lamb*(T[t-1,x+1] -2*T[t-1,x] + T[t-1,x-1])
        
        
        # --- Caso cond. fluxo no lado direito 
        #(A ultima temperatura é calculada depois da antepenultima, 
        # assim o fluxo do lado direito deve vir após o calculo das 
        # temperaturas ao longo da barra)
        if fluxo[1] == 1:
            T[t,-1] = T[t,-3] +2*dx*T_dir
    
# Impressão das temperaturas em cada instante        
#print('T = ',np.round(T[t,:],2))
        
        
# --- Recorte das posições auxiliares das cond. de fluxo
if fluxo[0] == 1: # lado esquerdo
    T = T[:, 1:]
if fluxo[1] == 1: # lado direito
    T = T[:,:-1]
    
# Impressão da matriz de temperaturas
#print("T_0 = ", T)

# ----------------------------------------------------------------- #
#                               PLOTS                               #
# ----------------------------------------------------------------- #
# --- MAPA DE CORES
fig, ax = plt.subplots()                      # Cria a figura com um subplot
xPlotCor = np.linspace(0,  L, len(T[0,:])+1); # Eixo x para o plot, sem 0 +1 não será mostrada a ultima coluna (direita)
tPlotCor = np.linspace(0, tf, len(T[:,0]));   # Eixo y para o plot
plt.pcolor(xPlotCor,tPlotCor,T, cmap='jet')   # Mapa de cores com eixos rotulados certos
#plt.pcolor(T, cmap='jet')                    # Mapa de cores com rótulos errados, indicando as 'posições' 'i' e 'j'
cbar = plt.colorbar()                         # Criando objeto barra de cores (para por rótulo)

# -- Rótulos
cbar.ax.set_ylabel('Temperatura (°C)', fontsize=16) # Rótulo da barra de cores
ax.set_ylabel('Tempo (s)', fontsize=16)             # Rótulo do eixo vertical
ax.set_xlabel('Comprimento (cm)', fontsize=16)      # Rótulo do eixo vertical

# -- Mostrar ou Salvar Figura
plt.show()
#savefig('Calor_Barra_-_Cores.pdf')


# --- CURVAS PARA INSTANTES DISTINTOS
fig, ax = plt.subplots() # Cria a figura com um subplot
plt.plot(X,T[  nt//5,:],                                   # 1/5 de tf
         X,T[2*nt//5,:],                                   # 2/5 de tf
         X,T[3*nt//5,:],                                   # 3/5 de tf
         X,T[4*nt//5,:],                                   # 4/5 de tf
         X,T[   nt-1,:])                                   # Instante Final tf

# -- Legendas
plt.legend([ 't = '+str(np.round(Inst[  nt//5],3))+' s',   # 1/5 de tf
             't = '+str(np.round(Inst[2*nt//5],3))+' s',   # 2/5 de tf
             't = '+str(np.round(Inst[3*nt//5],3))+' s',   # 3/5 de tf
             't = '+str(np.round(Inst[4*nt//5],3))+' s',   # 4/5 de tf
             't = '+str(np.round(Inst[   nt-1],3))+' s' ]) # Instante Final tf

# -- Rótulos
plt.xlabel('x [cm]', fontsize=16)
plt.ylabel('Temperatura [°C]', fontsize=16)

# -- Mostrar ou Salvar Figura
plt.show()
#savefig('Calor_Barra_-_Curvas.pdf')


Temperatura no lado esquerdo (digite d cond. na derivada): 100
 Temperatura no lado direito (digite d cond. na derivada): d
dT/dx no lado direito: 0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>